In [1]:
import sys
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

# 将 Projection 目录加入路径，以便导入 rellis_utils
# 假设当前 notebook 在 notebbooks/ 目录下，而 Projection 在上级目录
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "Projection"))
if project_root not in sys.path:
    sys.path.append(project_root)

# 导入现有的工具函数
try:
    from rellis_utils.lidar2img import load_from_bin, get_cam_mtx, get_mtx_from_yaml, points_filter
    print("成功导入 rellis_utils")
except ImportError as e:
    print(f"导入失败: {e}")
    print(f"请检查路径是否正确: {project_root}")

# 设置 matplotlib 显示参数
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)

成功导入 rellis_utils


In [11]:
import os
import glob

# ================= 配置区域 =================
# 1. Rellis-3D 数据集根目录
RELLIS_ROOT = '/home/xzy/datasets/Rellis-3D'

# 2. 推理结果目录
INFERENCE_DIR = '/home/xzy/Downloads/convertedRellis/rellisv3_edl_train-4/01_inferenced_npy'

# 3. 选择序列和帧号
# 修改点：根据你的 ls 输出，数据位于 00004 文件夹，而不是 00000
SEQ_ID = '00004' 
FRAME_ID = '000000' 

# ================= 自动生成文件路径 =================

# --- 1. 图像路径 (使用 glob 匹配时间戳) ---
img_pattern = os.path.join(RELLIS_ROOT, SEQ_ID, 'pylon_camera_node', f"frame{FRAME_ID}-*.jpg")
img_files = glob.glob(img_pattern)

if len(img_files) > 0:
    img_file = img_files[0]
else:
    print(f"警告: 未找到图像文件: {img_pattern}")
    img_file = None

# --- 2. LiDAR 点云路径 ---
# Rellis 的 bin 文件通常没有时间戳后缀，直接是 000000.bin
lidar_file = os.path.join(RELLIS_ROOT, SEQ_ID, 'os1_cloud_node_kitti_bin', f"{FRAME_ID}.bin")

# --- 3. 语义预测结果 (.npy) ---
npy_pattern = os.path.join(INFERENCE_DIR, SEQ_ID, f"frame{FRAME_ID}-*.npy")
npy_files = glob.glob(npy_pattern)

# --- 4. 标定文件路径 ---
camera_info_file = os.path.join(RELLIS_ROOT, 'Rellis_3D_cam_intrinsic', 'Rellis-3D', SEQ_ID, 'camera_info.txt')
if not os.path.exists(camera_info_file):
    camera_info_file = os.path.join(RELLIS_ROOT, SEQ_ID, 'camera_info.txt')

transforms_file = os.path.join(RELLIS_ROOT, 'Rellis_3D_cam2lidar_20210224', 'Rellis_3D', SEQ_ID, 'transforms.yaml')
if not os.path.exists(transforms_file):
    transforms_file = os.path.join(RELLIS_ROOT, SEQ_ID, 'transforms.yaml')

# ================= 验证输出 =================
print(f"当前测试序列: {SEQ_ID}, 帧号: {FRAME_ID}")
print("-" * 30)

# 检查图像
if img_file and os.path.exists(img_file):
    print(f"[OK] Image File: {img_file}")
else:
    print(f"[ERROR] Image File Not Found (Pattern: {img_pattern})")

# 检查 LiDAR
if os.path.exists(lidar_file):
    print(f"[OK] LiDAR File: {lidar_file}")
else:
    print(f"[ERROR] LiDAR File Not Found: {lidar_file}")

# 检查 NPY
if prob_map_file and os.path.exists(prob_map_file):
    print(f"[OK] Prob Map File: {prob_map_file}")
else:
    print(f"[ERROR] Prob Map File Not Found (Pattern: {npy_pattern})")

# 检查标定
print(f"[{'OK' if os.path.exists(camera_info_file) else 'ERROR'}] Camera Info: {camera_info_file}")
print(f"[{'OK' if os.path.exists(transforms_file) else 'ERROR'}] Transforms: {transforms_file}")

当前测试序列: 00004, 帧号: 000000
------------------------------
[OK] Image File: /home/xzy/datasets/Rellis-3D/00004/pylon_camera_node/frame000000-1581791678_408.jpg
[OK] LiDAR File: /home/xzy/datasets/Rellis-3D/00004/os1_cloud_node_kitti_bin/000000.bin
[OK] Prob Map File: /home/xzy/Downloads/convertedRellis/rellisv3_edl_train-4/01_inferenced_npy/00004/frame000000-1581791678_408.npy
[OK] Camera Info: /home/xzy/datasets/Rellis-3D/Rellis_3D_cam_intrinsic/Rellis-3D/00004/camera_info.txt
[OK] Transforms: /home/xzy/datasets/Rellis-3D/Rellis_3D_cam2lidar_20210224/Rellis_3D/00004/transforms.yaml
